In [12]:
def cluster_bert_clusters(final_scores, k):
    cluster_number = 0
    final_clusters = {}
    used_strings = {}
    for key in final_scores:
        quota = float(final_scores[key][0][0]) * 1/(k)
        difference = abs(final_scores[key][0][0] - final_scores[key][1][0])
            
        if ((len(final_scores[final_scores[key][1][1]]) > 1 and \
             key == final_scores[final_scores[key][1][1]][1][1] and difference < quota) or \
        (len(final_scores[final_scores[key][1][1]]) > 2 and \
        key == final_scores[final_scores[key][1][1]][2][1] and difference < (quota / 2)) or \
        (len(final_scores[final_scores[key][1][1]]) > 3 and \
        key == final_scores[final_scores[key][1][1]][3][1] and \
         (difference < (quota/3)))):        
            # Combine clusters if needed
            if key in used_strings and final_scores[key][1][1] in used_strings:
                cluster_number_one = used_strings[key]
                cluster_number_two = used_strings[final_scores[key][1][1]]
                if cluster_number_two != cluster_number_one:
                    final_clusters[cluster_number_one] = final_clusters[cluster_number_one] + final_clusters[cluster_number_two]
                    for string in final_clusters[cluster_number_two]:
                        used_strings[string] = cluster_number_one
                    final_clusters.pop(cluster_number_two, None)
            elif key in used_strings:
                final_clusters[used_strings[key]].append(final_scores[key][1][1])
            elif final_scores[key][1][1] in used_strings:
                final_clusters[used_strings[final_scores[key][1][1]]].append(key)
            else:
                used_strings[key] = cluster_number
                used_strings[final_scores[key][1][1]] = cluster_number
                final_clusters[cluster_number] = [key, final_scores[key][1][1]]
                cluster_number = cluster_number + 1
        else:
            # If its not similar add it to its own cluster - we'll cut all of length one later
            used_strings[key] = cluster_number
            final_clusters[cluster_number] = [key]
            cluster_number = cluster_number + 1
            
    return final_clusters

In [9]:
def create_bert_model(questions):
    from bert_serving.client import BertClient 
    
    # Start BERT client for mrpc bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/mrpc_output -num_worker=4 -max_seq_len=50
    # Start BERT client for qqa checkpoint bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/qqp_output -num_worker=4 -max_seq_len=50 -ckpt_name bert_model.ckpt-1000
    bc = BertClient(check_version=False)  
    doc_vecs = bc.encode(questions)
    return (bc, doc_vecs)

In [10]:
def find_k_similar_sentences(bc, doc_vecs, k, all_questions, question):
    import np
    
    query_vec = bc.encode([question])
    
    score = np.sum(query_vec * doc_vecs, axis=1)/np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:k]
    
    final_scores = {}
    
    for idx in topk_idx:
        final_scores.setdefault(question, []).append((score[idx], all_questions[idx]))
    
    return final_scores

In [11]:
def number_clusters(all_bert_clusters):
    
    curr_num = 1
    
    # Return cluster number -> node list and node -> cluster number
    final_bert_clusters = {}
    final_bert_cluster_locations = {}
    
    for cluster in all_bert_clusters:
        final_bert_clusters[curr_num] = cluster
        
        for sentence in cluster:
            final_burt_cluster_locations[sentence] = curr_num
            
        curr_num = curr_num + 1
    
    return(final_bert_clusters, final_bert_cluster_locations)
        